In [1]:
age_gender_bkts = pd.read_csv("age_gender_bkts.csv")
countries = pd.read_csv("countries.csv")
sessions = pd.read_csv("sessions.csv")
test_users = pd.read_csv("test_users.csv")
train_users_2 = pd.read_csv("train_users_2.csv")
sample_submission_NDF = pd.read_csv("sample_submission_NDF.csv")

In [2]:
def merge_Airbnb_data():
    
    test_users["country_destination"] = np.nan
    Airbnb = pd.concat([train_users_2,test_users], ignore_index=True, axis=0)
    Airbnb = pd.merge(Airbnb, sessions, left_on="id", right_on="user_id",how="left")
    Airbnb = pd.merge(Airbnb, countries, how="left", left_on="country_destination", right_on="country_destination")
    
    return Airbnb

Airbnb = merge_Airbnb_data()

In [3]:
def datetime_check():
    
    Airbnb["trans_timestamp_first_active"] = Airbnb["timestamp_first_active"].apply(lambda x: str(x)[:8])
    Airbnb["c_time"] = pd.to_datetime(Airbnb.date_account_created)
    Airbnb["b_time"] = pd.to_datetime(Airbnb.trans_timestamp_first_active)
    Airbnb["time_subs"] = Airbnb["c_time"] - Airbnb["b_time"]
    
    return Airbnb

Airbnb = datetime_check()

In [5]:
def data_transform():
    
    Airbnb["y"] = Airbnb["country_destination"].replace({'NDF':0, 'US':1, 'other':2, 'FR':3, 'CA':4, 'GB':5, 
                                                         'ES':6, 'IT':7, 'PT':8, 'NL':9,'DE':10, 'AU':11, np.nan:12})
    Airbnb["signup_app_transform"] = Airbnb["signup_app"].replace({'Web':0, 'Moweb':1, 'iOS':2, 'Android':3})
    Airbnb["signup_method_transform"] = Airbnb["signup_method"].replace({'facebook':0,'basic':1,'google':2,'weibo':3})
    Airbnb["time_subs_change"] = Airbnb["time_subs"].astype(str)
    Airbnb["time_subs_change"] = Airbnb["time_subs_change"].replace(" days 00:00:00.000000000" , "", regex=True)
    Airbnb["time_subs_change"] = Airbnb["time_subs_change"].astype(float)
    
    c = Airbnb.filter(items=["signup_app_transform", "signup_method_transform", "time_subs_change"])
    
    return Airbnb, c

Airbnb, c = data_transform()

In [6]:
from sklearn.metrics import *
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

def test_classification():
    
    #qda
    qda = QuadraticDiscriminantAnalysis(store_covariance=True).fit(c, Airbnb["y"])
    
    y_true = Airbnb["y"]
    y_pred = qda.predict(c)
    
    #lda
    lda = LinearDiscriminantAnalysis(n_components=3, solver="svd", store_covariance=True).fit(c, Airbnb["y"])
    
    y_true_l = Airbnb["y"]
    y_pred_l = lda.predict(c)

    
    print("QDA"), print('\n'), print('qda_accuracy_score =', accuracy_score(Airbnb['y'], qda.predict(c))), \
   #print(confusion_matrix(y_true, y_pred)), \ 
    print('\n'), print(classification_report(y_true, y_pred)), print('\n'), \
    
    print("========================================================================"), print('\n'),\
    
    print("LDA"), print('\n'), print('lda_accuracy_score =', accuracy_score(Airbnb['y'], lda.predict(c))), \
   #print(confusion_matrix(y_true_l, y_pred_l)),\
    print('\n'), print(classification_report(y_true_l, y_pred_l))

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
test_classification() #QDA, LDA

QDA


qda_accuracy_score = 0.0005908198069857463


             precision    recall  f1-score   support

          0       0.00      0.00      0.00   3137212
          1       0.00      0.00      0.00   1717805
          2       0.00      0.00      0.00    327603
          3       0.00      0.00      0.00    149748
          4       0.00      0.00      0.00     40286
          5       0.00      0.00      0.00     66624
          6       0.00      0.00      0.00     68716
          7       0.00      0.00      0.00    103607
          8       0.00      1.00      0.00      6306
          9       0.00      0.00      0.00     23083
         10       0.00      0.00      0.00     24202
         11       0.00      0.00      0.00     12401
         12       0.00      0.00      0.00   4995712

avg / total       0.00      0.00      0.00  10673305





LDA


lda_accuracy_score = 0.4680580195169163


             precision    recall  f1-score   support

          0       0.31      0.00      0.00   

In [9]:
import xgboost
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

def test_xgboost():
    x = c[:555555]
    y = Airbnb["y"][:555555]
    model = XGBClassifier()
    model.fit(x,y)
    
    print(model)
    print('\n')
    print("XGBoost")
    print('\n')
    print("test_xgboost_accuracy_score =", accuracy_score(y, model.predict(x)))
    print('\n')
    print(classification_report(y, model.predict(x)))
    print('\n')
   #print(confusion_matrix(y, model.predict(x)))

In [10]:
test_xgboost() #XGBoost

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)


XGBoost


test_xgboost_accuracy_score = 0.5723357723357724


             precision    recall  f1-score   support

          0       0.57      1.00      0.73    317902
          1       0.64      0.00      0.00    168414
          2       1.00      0.00      0.00     28550
          3       0.00      0.00      0.00     16725
          4       0.00      0.00      0.00      2083
          5       0.00      0.00      0.00      4312
          6       0.00      0.00      0.00      4514
          7       0.00      0.00      0.00      7518
          8       0.00      0.00      0.00       217
          9       0